Date : 03/27/2022<br />
Author : Xueying Li<br />
Description : This file is used to test and evaluate the model's accuracy<br />
Reference: I used part of code from https://github.com/mazabdul7/AtTheEdge/blob/main/Training.ipynb.<br /> 

In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pathlib
from tensorflow.keras import layers
import random
from sklearn.metrics import confusion_matrix


batch_size = 16
modelCheckPointName='bestmodel95_79_224.hdf5'


In [2]:
def createModel(modelCheckPointName,batch_size,basemodel_img_height=224,basemodel_img_width=224):
    #Augment training data to reduce overfitting by creating a pre-proccessing layer that only activates in training
    data_augmentation = keras.Sequential(
        [tf.keras.layers.RandomFlip("horizontal"),tf.keras.layers.RandomFlip("vertical"),tf.keras.layers.RandomRotation(0.5)]
    )
    #Load EfficientNet base or MobileNet with weights pre-trained on ImageNet
    base_model =keras.applications.MobileNetV3Large(input_shape=(basemodel_img_height, basemodel_img_width, 3),include_top=False,weights='imagenet')
    # Create new model on top
    inputs = keras.Input(shape=(384, 512, 3))
    # Apply random data augmentation
    x = data_augmentation(inputs)  
    # Resize layer to basemodel inputsize (only used in downscal=ling)

    x=tf.keras.layers.Resizing(basemodel_img_width,basemodel_img_height)(x)
    base_model.trainable = True
    x = base_model(x, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    #There are seven class
    outputs = keras.layers.Dense(7, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.summary()
    model.load_weights(modelCheckPointName)
    model.layers[3].trainable = False
    
    model.summary()
    return model

In [3]:
def generateTrainingData(train,batch_size):

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      train,
      image_size=(384,512),
      batch_size=batch_size)

    #Print number of samples
    print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
    class_names = train_ds.class_names
    train_ds = train_ds.cache().prefetch(buffer_size=10)
    return train_ds

In [4]:
#Import test set as images
def testAccuracy(testDataName,label,model):
    model=model
    labels = ["cardboard", "empty","glass","hand","metal", "paper", "plastic"]
    wrongCount = 0
    predictedLabel = []
    correctLabel=[]
    wrongImage = []
    img = keras.preprocessing.image.DirectoryIterator(
        testDataName, tf.keras.preprocessing.image.ImageDataGenerator(), target_size=(384, 512), batch_size=1,shuffle=False)
    for index in range(len(img)):
        predictions = model.predict(img[index][0])
        predictedLabel.append(np.argmax(predictions))
        for j in img[index][1]: 
            if np.argmax(predictions) != j.tolist().index(1):
                wrongCount+=1
                wrongImage.append(img[index])
                plt.figure(figsize=(5,5))
                plt.imshow(((1-img[index][0][0])*255).astype('uint8'))
                plt.show()
        correctLabel.append(j.tolist().index(1))
    print("The test accuracy is " + str(((len(img)-wrongCount)/len(img))*100) + "%")
    cm = confusion_matrix(correctLabel,predictedLabel)
    cm =cm/cm.sum(axis=1)
    cm=np.mean(cm.diagonal())
    print("The perclass accuracy is " + str(cm) + "%")
    return ((len(img)-wrongCount)/len(img))*100
labels = ["cardboard", "glass","metal", "paper", "plastic"]
#wrongImage=testAccuracy("TOPtest_",labels,model)
#print(wrongImage)
#wrongImag1e=testAccuracy("TOPtest2","Confusion matrix (top1)",labels,model)


In [18]:
def trainModel(tran_ds,model,epochs,learningrate):
    model.compile(optimizer=keras.optimizers.Adam(learningrate),  # Low learning rate identified by lowest loss within the first 5 epochs
        loss="sparse_categorical_crossentropy" ,
        metrics=["accuracy"],
    )

    model.fit(train_ds, epochs=epochs)
    return model

In [28]:
def plotAccuracy(n_accuracy,o_accuracy):
    plt.plot(list(range(33)),n_accuracy,label="new data test acccuracy",marker=".")
    plt.plot(list(range(33)),o_accuracy,label="old data test acccuracy",marker=".")
    plt.xlabel("epochs")
    plt.ylabel("Accuracy")
    plt.grid()
    plt.legend()
    plt.title("Accuracy on new data and original data")
    plt.show

In [43]:
!jupyter nbconvert --to python learning.ipynb

[NbConvertApp] Converting notebook learning.ipynb to python
[NbConvertApp] Writing 10163 bytes to learning.py


In [25]:
from sklearn.metrics import confusion_matrix
def cm(correctLabel,predictedLabel,titleName,labels=["cardboard", "glass","metal", "paper", "plastic"]):
    confusionmatrix = confusion_matrix(correctLabel,predictedLabel)
    cm= []
    cm = confusionmatrix/confusionmatrix.sum(axis=1)
    print(cm)
    cm=np.mean(cm.diagonal())
    print(cm)
    # Make the confusion matrix plot
    fig, axes = plt.subplots()
    axes.imshow(confusionmatrix, interpolation='nearest', cmap = plt.cm.GnBu)
    axes.set_title(titleName)
    n = np.array(range(len(labels)))
    axes.set_yticks(n)
    axes.set_yticklabels(labels)
    axes.set_xticks(n)
    axes.set_xticklabels(labels, rotation=45)
    # 
    threshold = confusionmatrix.max() / 2. 
    for i in range(len(confusionmatrix)):
        for j in range(len(confusionmatrix[0])):
            axes.text(j, i, format(confusionmatrix[i, j], 'd'),horizontalalignment = 'center', 
                                   color = 'white' if confusionmatrix[i, j] > threshold else "black")
        axes.set_ylabel('True label')
        axes.set_xlabel('Predicted label')


In [54]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 384, 512, 3)]     0         
                                                                 
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 960)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 7)                 6727      
                                                                 
Total params: 3,003,079
Trainable params: 2,978,679
Non-train